
# PANGAEA Bench Hands-on Tutorial

## 🌟 What This Colab File Provides
This notebook serves as a quickstart guide for setting up and running experiments with PANGAEA's benchmarking framework. It includes:
1. **Environment Setup**: Clone the repository and install required dependencies.
2. **Data Loading**: Instructions on how to load datasets for geospatial tasks.
3. **Model Selection and Training**: Select a model and configure a pipeline for training.
4. **Running the Pipeline**: Complete an end-to-end experiment for evaluation.

Follow along to get started!

But before REMEMBER TO CHANGE THE RUNTIME TYPE to GPU

## Clone the Repository

First clone the PANGAEA repository and navigate to the directory:

In [ ]:
!git clone https://github.com/VMarsocci/pangaea-bench.git
%cd pangaea-bench

Cloning into 'pangaea-bench'...
remote: Enumerating objects: 4667, done.
remote: Counting objects: 100% (899/899), done.
remote: Compressing objects: 100% (354/354), done.
remote: Total 4667 (delta 695), reused 549 (delta 544), pack-reused 3768 (from 3)
Receiving objects: 100% (4667/4667), 3.14 MiB | 27.92 MiB/s, done.
Resolving deltas: 100% (3116/3116), done.
/content/pangaea-bench


## Install Dependencies
   We will set up the environment using pip.

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install imagecodecs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 MB 24.6 MB/s eta 0:00:00


In [ ]:
# Probably you had to restart the runtime to correctly install packages, so let's change path again
%cd pangaea-bench
# Install the repository as a development package
!pip install --no-build-isolation --no-deps -e .

/content/pangaea-bench
Obtaining file:///content/pangaea-bench
  Preparing metadata (setup.py) ... done
  Running setup.py develop for pangaea


## Adding a new dataset

We will implement a *toy* version of [FLAIR dataset](https://arxiv.org/pdf/2211.12979). The data are already stored on [Google Drive](https://drive.google.com/drive/folders/1uAJ7-m7stdeGenKQMLkRaEkKhR4uVbzU?usp=sharing)

In [ ]:
#let's download the dataset from this link https://drive.google.com/file/d/1U8woGV2YlzeGSZKN2BPdz4gAySkEsmiJ/view?usp=drive_link

!gdown --id 1U8woGV2YlzeGSZKN2BPdz4gAySkEsmiJ --output toyFLAIR.zip

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1U8woGV2YlzeGSZKN2BPdz4gAySkEsmiJ
From (redirected): https://drive.google.com/uc?id=1U8woGV2YlzeGSZKN2BPdz4gAySkEsmiJ&confirm=t&uuid=80e67622-6e8c-411f-acbe-5491886b3e99
To: /content/pangaea-bench/toyFLAIR.zip
100% 226M/226M [00:01<00:00, 117MB/s]


In [ ]:
!unzip toyFLAIR.zip

Archive:  toyFLAIR.zip
   creating: toyFLAIR/
   creating: toyFLAIR/test/
   creating: toyFLAIR/test/images/
  inflating: toyFLAIR/test/images/IMG_061946.tif  
  inflating: toyFLAIR/test/images/IMG_062207.tif  
  inflating: toyFLAIR/test/images/IMG_062393.tif  
  inflating: toyFLAIR/test/images/IMG_062524.tif  
  inflating: toyFLAIR/test/images/IMG_062871.tif  
  inflating: toyFLAIR/test/images/IMG_063104.tif  
  inflating: toyFLAIR/test/images/IMG_063120.tif  
  inflating: toyFLAIR/test/images/IMG_063366.tif  
  inflating: toyFLAIR/test/images/IMG_063620.tif  
  inflating: toyFLAIR/test/images/IMG_063812.tif  
  inflating: toyFLAIR/test/images/IMG_064026.tif  
  inflating: toyFLAIR/test/images/IMG_064027.tif  
  inflating: toyFLAIR/test/images/IMG_064234.tif  
  inflating: toyFLAIR/test/images/IMG_064890.tif  
  inflating: toyFLAIR/test/images/IMG_065011.tif  
  inflating: toyFLAIR/test/images/IMG_065120.tif  
  inflating: toyFLAIR/test/images/IMG_065229.tif  
  inflating: toyFLAIR/te

The data structure is as follows:
```
toyFLAIR/
├── train/
│   ├── images/
│   └── labels/
├── val/
│   ├── images/
│   └── labels/
└── test/
    ├── images/
    └── labels/
```

How to add a new dataset? Follow [these instructions](https://github.com/VMarsocci/pangaea-bench/blob/main/.github/CONTRIBUTING.md#adding-a-new-downstream-dataset)

TL;DR:
you need to create two files:
 - a _config.yaml_ file and put it in _configs/dataset_
 - a _dataset.py_ file and put in _pangaea/dataset_


Follow this code for the _dataset.py_ file:

In [ ]:
import torch
from pangaea.datasets.base import RawGeoFMDataset

class MyDataset(RawGeoFMDataset):
     def __init__(
        self,
        split: str,
        dataset_name: str,
        multi_modal: bool,
        multi_temporal: int,
        root_path: str,
        classes: list,
        num_classes: int,
        ignore_index: int,
        img_size: int,
        bands: dict[str, list[str]],
        distribution: list[int],
        data_mean: dict[str, list[str]],
        data_std: dict[str, list[str]],
        data_min: dict[str, list[str]],
        data_max: dict[str, list[str]],
        download_url: str,
        auto_download: bool,
        temp: int, #newly added parameter
    ):
        super(MyDataset, self).__init__(
            split=split,
            dataset_name=dataset_name,
            multi_modal=multi_modal,
            multi_temporal=multi_temporal,
            root_path=root_path,
            classes=classes,
            num_classes=num_classes,
            ignore_index=ignore_index,
            img_size=img_size,
            bands=bands,
            distribution=distribution,
            data_mean=data_mean,
            data_std=data_std,
            data_min=data_min,
            data_max=data_max,
            download_url=download_url,
            auto_download=auto_download,
        )

        self.temp = temp #newly added parameter
        # Initialize file lists or data structures here

    def __len__(self):
        # Return the total number of samples
        return len(self.file_list)

    def __getitem__(self, index):
       """Returns the i-th item of the dataset.

       Args:
           i (int): index of the item

       Raises:
           NotImplementedError: raise if the method is not implemented

       Returns:
           dict[str, torch.Tensor | dict[str, torch.Tensor]]: output dictionary follwing the format
           {"image":
               {
               "optical": torch.Tensor of shape (C T H W) (where T=1 if single-temporal dataset),
                "sar": torch.Tensor of shape (C T H W) (where T=1 if single-temporal dataset),
                },
           "target": torch.Tensor of shape (H W) of type torch.int64 for segmentation, torch.float for
           regression datasets.,
            "metadata": dict}.
       """
        # Load your data and labels here
        image = ...  # Load image
        target = ...  # Load target label or mask

        # Convert to tensors
        image = torch.tensor(image, dtype=torch.float32)
        target = torch.tensor(target, dtype=torch.long)

        return {
            'image': {'optical': image},
            'target': target,
            'metadata': {}
        }

    @staticmethod
    def download(self, silent=False):
        # Implement if your dataset requires downloading
        pass

Some other practical hints/reccomendetions:
 - name your _dataset_ file: ***toy_flair.py*** and put it in _configs/dataset_
 - name your _config_ file: ***toyflair.yaml*** and put it in _pangaea/datasets_
 - HINT! Get inspired by FiveBillionPixels files to follow the structure of the files. The data structure is the same!

Now you are ready to run the code on your newly added dataset. Run the following command line!

In [ ]:
# Name of the config referring to the dataset
# Model: RemoteCLIP (lightweight, checkpoints downloaded automatically)
# Decoder: UperNet
# Preprocessing and criterion

!torchrun --nnodes=1 --nproc_per_node=1 pangaea/run.py \
  --config-name=train \
  dataset=toyflair \
  encoder=remoteclip \
  decoder=seg_upernet \
  preprocessing=seg_default \
  criterion=cross_entropy \
  task=segmentation

## Adding a new model

We will implement [FG-MAE](https://github.com/zhu-xlab/FGMAE). The checkpoints (ViT-S) are already stored on [Google Drive](https://drive.google.com/file/d/14EJgdtyIJ9Gk8A4ritoF6JPkeCZmCeS_/view?usp=sharing)

In [ ]:
#let's download the dataset from this link https://drive.google.com/file/d/14EJgdtyIJ9Gk8A4ritoF6JPkeCZmCeS_/view?usp=drive_link

!gdown --id 14EJgdtyIJ9Gk8A4ritoF6JPkeCZmCeS_ --output pretrained_models/B13_vits16_fgmae_ep99.pth

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=14EJgdtyIJ9Gk8A4ritoF6JPkeCZmCeS_
From (redirected): https://drive.google.com/uc?id=14EJgdtyIJ9Gk8A4ritoF6JPkeCZmCeS_&confirm=t&uuid=c4426569-0fa8-41aa-aa7d-67449d65c01e
To: /content/pangaea-bench/pretrained_models/B13_vits16_fgmae_ep99.pth
100% 585M/585M [00:28<00:00, 20.4MB/s]


How to add a new model? Follow [these instructions](https://github.com/VMarsocci/pangaea-bench/blob/main/.github/CONTRIBUTING.md#adding-a-new-geospatial-foundation-model)

TL;DR:
you need to create two files:
 - a _config.yaml_ file and put it in _configs/encoder_
 - a _model.py_ file and put in _pangaea/encoders_


Follow this code for the _model.py_ file:

In [ ]:
import torch.nn as nn

from pangaea.encoders.base import Encoder

class MyModel(Encoder):
    def __init__(
        self,
        encoder_weights: str | Path,
        input_size: int,
        input_bands: dict[str, list[str]],
        output_layers: int | list[int],
        in_chans: int,              #newly added parameter
    ) -> None:
        super().__init__(
            model_name="my_model_name",
            encoder_weights=encoder_weights,
            input_bands=input_bands,
            input_size=input_size,
            embed_dim=768,        # my_model_embed_dim, fixed parameters
            output_dim=768,       # my_model_output_dim, fixed parameters
            multi_temporal=False, # wether support multi-temporal, fixed parametersfixed parameters
            multi_temporal_ouput=False, # wether the output of the model has a temporal dimension
        )

       self.in_chans = in_chans    #newly added parameter

        # Initialize your model architecture here
        # For example:
        self.backbone = nn.Sequential(
            nn.Conv2d(in_chans, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            # Add more layers as needed
        )
        # Specify output layers if applicable

    def load_encoder_weights(self, pretrained_path: str) -> None:
        # Load pretrained weights
        state_dict = torch.load(pretrained_path, map_location='cpu')
        self.load_state_dict(state_dict, strict=False)
        print(f"Loaded encoder weights from {pretrained_path}")

    def forward(self, x: dict[str, torch.Tensor]) -> list[torch.Tensor]:
        """Foward pass of the encoder.

        Args:
            x (dict[str, torch.Tensor]): encoder's input structured as a dictionary:
            x = {modality1: tensor1, modality2: tensor2, ...}, e.g. x = {"optical": tensor1, "sar": tensor2}.
            If the encoder is multi-temporal (self.multi_temporal==True), input tensor shape is (B C T H W) with C the
            number of bands required by the encoder for the given modality and T the number of time steps. If the
            encoder is not multi-temporal, input tensor shape is (B C H W) with C the number of bands required by the
            encoder for the given modality.

        Returns:
            list[torch.Tensor]: list of the embeddings for each modality. For single-temporal encoders, the list's
            elements are of shape (B, embed_dim, H', W'). For multi-temporal encoders, the list's elements are of shape
            (B, C', T, H', W') with T the number of time steps if the encoder does not have any time-merging strategy,
            else (B, C', H', W') if the encoder has a time-merging strategy (where C'==self.output_dim).
        """
        x = image['optical']
        outputs = []
        # Forward pass through the model
        for idx, layer in enumerate(self.backbone):
            x = layer(x)
            if idx in self.output_layers:
                outputs.append(x)
        return outputs

Some other practical hints/reccomendetions:
 - name your _model_ file: ***fgmae_encoder.py*** and put it in _model/encoders_
 - name your _config_ file: ***fgmae.yaml*** and put it in _configs/encoder_
 - HINT! Get inspired by SSL4EO_MAE files to follow the structure of the files. The  structure is the same! Also this [link](https://github.com/zhu-xlab/FGMAE/blob/main/src/transfer_classification/models/mae/models_vit.py) can be useful!

Now you are ready to run the code on your newly added model. Run the following command line!

In [ ]:
# Use torchrun to launch training
# Dataset: toyflair (already added)
# Encoder: FG-MAE model
# Decoder: UPerNet
# Preprocessing: default settings
# Criterion: cross entropy loss
# Task: segmentation
!torchrun --nnodes=1 --nproc_per_node=1 pangaea/run.py \
  --config-name=train \
  dataset=toyflair \
  encoder=fgmae \
  decoder=seg_upernet \
  preprocessing=seg_default \
  criterion=cross_entropy \
  task=segmentation


INFO - 04/29/25 10:01:33 - 0:00:00 - ============ Initialized logger ============
INFO - 04/29/25 10:01:33 - 0:00:00 - 'batch_size': 8,
                                      'ckpt_dir': None,
                                      'criterion': {'_target_': 'torch.nn.CrossEntropyLoss',
                                                    'ignore_index': '${dataset.ignore_index}'},
                                      'data_replicate': 1,
                                      'dataset': {'_target_': 'pangaea.datasets.toy_flair.ToyFLAIR',
                                                  'auto_download': False,
                                                  'bands': {'optical': ['B4', 'B3', 'B2']},
                                                  'classes': ['building', 'pervious surface', 'impervious surface',
                                                              'bare soil', 'water', 'coniferous', 'deciduous',
                                                              'bru